In [1]:
from lib.indicators.rsi import RSI
from lib.indicators.mfi import MFI
from lib.indicators.macd import MACD
from lib.indicators.nvi import NVI
from lib.indicators.koncorde import KONCORDE
from lib.indicators.ema import EMA

indicators = [
    MACD(23,12,9),
    RSI(30, 70, 14),
    EMA(14),
    MFI(20, 80, 14),
    NVI(255),
    KONCORDE(15)
]

In [2]:
from lib.strategies.RLstrategy import RL

2024-03-17 21:37:59.707221: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-17 21:37:59.709484: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-17 21:37:59.740311: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 21:37:59.740341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 21:37:59.741109: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
strategy = RL(indicators)

2.15.0


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime
# Binance Client
try:
  from binance.client import Client as BinanceProvider
except ModuleNotFoundError:
  !pip install python-binance
  from binance.client import Client as BinanceProvider

# Define the ticker symbol for Bitcoin (BTC-USD)
ticker = "BTCUSDT"

# Download historical data using yfinance
api_key = "OF6SkzXI0EAcvmMWlkeUKl6YyxYIFU4pN0Bj19gaVYZcgaTt7OImXxEyvoPcDhmk"
secret_key = "tXay1BDYuSyigxvl27UQIBJbIHADaep8FT7HPO9Mb3vfmcyDkz4keEaHkpm7dcFe"
provider= BinanceProvider(api_key, secret_key, tld='us')

klines = provider.get_historical_klines(ticker,BinanceProvider.KLINE_INTERVAL_1MINUTE, "15 Mar, 2024")
bitcoin_data = pd.DataFrame(klines, columns = ["Open time", "Open", "High", "Low", "Close", "Volume", "Close time", "Quote asset volume"," Number of trades"," Taker buy base asset volume", "Taker buy quote asset volume", "Ignore"])
bitcoin_data['Date'] = bitcoin_data['Open time'].apply(lambda x : datetime.fromtimestamp(x / 1000).strftime('%Y-%m-%d %H:%M:%S'))
bitcoin_data['Close'] =  bitcoin_data['Close'].apply(lambda x : float(x))

bitcoin_data = bitcoin_data.drop('Open time', axis=1)
bitcoin_data = bitcoin_data.set_index("Date")

# Display the first few rows of the data
print(bitcoin_data.head())
bitcoin_data.info()


                               Open            High             Low     Close  \
Date                                                                            
2024-03-14 21:00:00  71405.00000000  71477.72000000  71405.00000000  71477.72   
2024-03-14 21:01:00  71528.42000000  71528.42000000  71471.11000000  71528.33   
2024-03-14 21:02:00  71462.11000000  71528.30000000  71462.11000000  71528.25   
2024-03-14 21:03:00  71528.16000000  71528.16000000  71469.00000000  71528.13   
2024-03-14 21:04:00  71534.99000000  71564.20000000  71504.56000000  71564.17   

                         Volume     Close time Quote asset volume  \
Date                                                                
2024-03-14 21:00:00  0.01212000  1710460859999       865.94345760   
2024-03-14 21:01:00  0.00638000  1710460919999       456.34962570   
2024-03-14 21:02:00  0.00166000  1710460979999       118.69060540   
2024-03-14 21:03:00  0.00061000  1710461039999        43.63098600   
2024-03-14 21:04:0

In [5]:
import numpy as np

bitcoin_data_subset =  bitcoin_data[["Open", "High", "Low", "Close","Volume"]].copy()
bitcoin_data_subset["r"] = np.log(bitcoin_data_subset["Close"] / bitcoin_data_subset["Close"].shift(1))
bitcoin_data_subset = bitcoin_data_subset.rename_axis('Date')
bitcoin_data_subset.index = pd.to_datetime(bitcoin_data_subset.index)

bitcoin_data_subset.info()

df_except_last = bitcoin_data_subset.iloc[:-1]

# DataFrame con solo la última fila
df_last = bitcoin_data_subset.iloc[-1]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4359 entries, 2024-03-14 21:00:00 to 2024-03-17 21:38:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    4359 non-null   object 
 1   High    4359 non-null   object 
 2   Low     4359 non-null   object 
 3   Close   4359 non-null   float64
 4   Volume  4359 non-null   object 
 5   r       4358 non-null   float64
dtypes: float64(2), object(4)
memory usage: 238.4+ KB


In [6]:
strategy.prepare_data(df_except_last)

/home/ale/Documentos/trabajo_profesional_algo_trading/algo_trader/lib/strategies/RLstrategy.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["High"] = self.data["High"].apply(lambda x: float(x))
/home/ale/Documentos/trabajo_profesional_algo_trading/algo_trader/lib/strategies/RLstrategy.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["Low"] = self.data["Low"].apply(lambda x: float(x))
/home/ale/Documentos/trabajo_profesional_algo_trading/algo_trader/lib/strategies/RLstrategy.py:3

In [7]:
strategy.predict(df_last)

[MACD] Current value: -26.44
[MACD] Signal: 0
[RSI] Current value: 30.7240568800635
[RSI] Sell Threshold: 70
[RSI] Buy Threshold: 30
[RSI] Signal: 0
[EMA] Current EMA value: 67816.70727669005
[EMA] Current Close value: nan
[EMA] Signal: 0
[MFI] Current value: 19.18500972061628
[MFI] Sell Threshold: 80
[MFI] Buy Threshold: 20
[MFI] Signal: 0
[NVI] Current nvi value: 1000.000019911659
[NVI] Current nvi_ema value: 1000.0000273885878
[NVI] Signal: 0
[KONCORDE] Small hands value: nan
[KONCORDE] Big hands value: -2.673355416554741
[KONCORDE] Trend value: nan
[KONCORDE] Trend avg value: 12.71927926531876
[KONCORDE] Signal: 0
1/1 [==============================] - 0s 152ms/step
signal Action.HOLD


<Action.HOLD: 'hold'>